# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [5]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [4]:
len(dataset["train"])

650000

In [5]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

,label,text
0,5 stars,"Love this place! I had a coupon for buy one entree get one free, but the bar code wasn't on my email & I couldn't get one to pull up on my phone. The manager took the meal off our bill anyway, even though we were going to eat there anyway. And the food was awesome as always."
1,4 stars,"I'm so happy I finally made it to Joyride last week!! The patio is awesome!!! The restaurant has a unique vibe that draws you in and makes you smile! The food is pretty good. We're not talking gourmet here - its simple eating, but the flavors are good and its definitely worth checking out. The Mexican Street Corn is AWESOME!! This is definitely one of my new weekend spots for some daytime fun!!"
2,2 star,"Ok Teharu..... it took nearly 40 minutes to be seated. Had we been called as our name appeared on the sign up sheet, it would have only been about a 20 minute wait. \nYou can't have a guy that speaks little english in charge of seating people if he is going to skip them/ and or give away the table to another party. \nThis was not an impressive visit. Better luck next time."
3,1 star,"Marti L.'s review is spot on.\n\nMy husband and I took our kids last year, they were 2 and 4 at the time. I'm writing this review now after being reminded of this place by my sister in law who wants to have her son's bday party here.\n\nFirst, the pics on the website are totally misleading.\n\nBasically, older kids will be totally bored, it is overpriced, the bounce house was FILTHY, the \""petting zoo\"" was a bunch of creepy reptiles that looked like they weren't supposed to be there. Seriously, a crocodile with its mouth taped up with duct tape? My kids were terrified! And I was concerned about the health and safety of the animals.\n\nThe normal animals you would expect to see are kept in pens around the periphery and you cannot pet them. You wouldn't want to anyway. The animals and their pens are filthy. \n\nThe \""hayride\"" was the worst part. It takes you around the edge of the farm where you get a view of garbage piles and neighboring buildings.\n\nMy kids did enjoy decorating the pumpkins, although it consisted of just sticking foam stickers on the pumpkins.\n\nWon't be back."
4,1 star,"Waste of time coming here for lunch! Came in with three co-workers on our lunch break and service was horrific! We immediately asked for extra bean dip. Our waitress, Stephanie, took our drink order and said she'd be back to take our food orders even tho we were ready to order right then. She proceeded to go behind the bar and make some drinks and ring some orders and check on other customers. She finally came back with our drinks but no bean dip. We waited and waited and waited for her to come take our orders and finally had to ask to get someone else to serve us. This turned out to be the manager. We had ordered four lunch specials and one to go. After waiting some more she came back and said we could get the to go order only at full price so we declined. We finally got more chips and salsa from our original waitress who brought two of everything even attitude. By this time we had been waiting for at least 30 min since the manager had supposedly put our order in. We got up and asked for the manager again since at this point we were running late for work. She's nowhere to be found until we get up to leave. She comes out with our food as we're half way out the door saying \""they tried their best\""! No apology. She offered to box it up but for what full price since the special is full price to go!? No thanks cuz they probably spit in it anyway!"
5,3 stars,"If you haven't experienced it, definitely recommend it!! Loved the food and the service. The prices are a bit high, but used a birthday voucher for $30 to supplement. (You can only sign up at the restaurant). Go early to avoid a long wait. Be sure to get the fried rice, it is worth it."
6,5 stars,Yummy falafel and hummus!
7,5 stars,"If you are at the Forum Shops and need something Gucci....go to P

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [8]:
pip install transformers

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"Wanted to love this place.... Fast, close to home, and great Yelp reviews. Alas, it was not to be...\nGot the Red Curry lunch with veggies, tofu, and brown rice. They had a great variety of veggies and they were cooked perfectly. Not limp, crispy without being too raw. Brown rice was perfectly cooked as well. Trouble was the curry. There was a layer of sauce on the veggies, as opposed to it swimming in curry. Which would have been fine except that there was a gritty texture, no silky coconut milk, no hint of fish sauce, and NO FLAVOR. There was not even a hint of heat, nor any other discernible spice notes. When has a curry ever been bland? I'm the wrong person to ask. I guess.\nI also tried the chicken pot stickers. They wrappers had a weird texture, the filling was super bland, and the sauce they came with was just thin, sweet, and totally forgettable. \n\nDecor was inviting, it seemed clean, and the staff seemed fine (ordered carry out, my order was waiting for me when I got there, both the guy on the phone and the gal on the register seemed competent & friendly-ish)...\nI may be back the next time my son bugs me for Pei Wei. :)","[101, 20601, 1106, 1567, 1142, 1282, 119, 119, 119, 119, 13227, 117, 1601, 1106, 1313, 117, 1105, 1632, 15821, 1233, 1643, 3761, 119, 2586, 2225, 117, 1122, 1108, 1136, 1106, 1129, 119, 119, 119, 165, 183, 2349, 3329, 1103, 2156, 20422, 5953, 1114, 1396, 9705, 1905, 117, 1106, 14703, 117, 1105, 3058, 7738, 119, 1220, 1125, 170, 1632, 2783, 1104, 1396, 9705, 1905, 1105, 1152, 1127, 13446, 6150, 119, 1753, 15905, 117, 19501, 1183, 1443, 1217, 1315, 7158, 119, 2671, 7738, 1108, 6150, 13446, 1112, 1218, 119, 17610, 1108, 1103, 16408, 6234, 119, 1247, 1108, 170, 6440, 1104, 14313, 1113, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(5000))

In [11]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

In [12]:
small_eval_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

### 数据全量

使用 全部 的数据样本，在 BERT 上演示大规模训练（基于 Pytorch Trainer）

In [13]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [14]:
full_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 650000
})

In [15]:
full_eval_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [17]:
!pip install transformers[torch]

In [7]:
!pip install accelerate -U

In [9]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.25.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
b

In [10]:
!pip install pytorch

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [18]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [19]:
!ls -l /usr/local/lib/python3.10/dist-packages/transformers/models/bert

total 344
-rw-r--r-- 1 root root 10150 Jan  4 14:41 configuration_bert.py
-rw-r--r-- 1 root root 10490 Jan  4 14:41 convert_bert_original_tf2_checkpoint_to_pytorch.py
-rw-r--r-- 1 root root  2159 Jan  4 14:41 convert_bert_original_tf_checkpoint_to_pytorch.py
-rw-r--r-- 1 root root  4098 Jan  4 14:41 convert_bert_pytorch_checkpoint_to_original_tf.py
-rw-r--r-- 1 root root  7606 Jan  4 14:41 convert_bert_token_dropping_original_tf2_checkpoint_to_pytorch.py
-rw-r--r-- 1 root root  6057 Jan  4 14:41 __init__.py
-rw-r--r-- 1 root root 84063 Jan  4 14:41 modeling_bert.py
-rw-r--r-- 1 root root 63600 Jan  4 14:41 modeling_flax_bert.py
-rw-r--r-- 1 root root 85690 Jan  4 14:41 modeling_tf_bert.py
drwxr-xr-x 2 root root  4096 Jan  4 14:41 __pycache__
-rw-r--r-- 1 root root 14883 Jan  4 14:41 tokenization_bert_fast.py
-rw-r--r-- 1 root root 25175 Jan  4 14:41 tokenization_bert.py
-rw-r--r-- 1 root root 11757 Jan  4 14:41 tokenization_bert_tf.py


下面这个代码。如果执行报错，重启colab会话重新执行即可

重启后需要重新执行下前面的命令 重新获取model、dataset、tokenized_datasets、small_train_dataset、small_eval_dataset



In [20]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [21]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_length=False,
half_precision_backend=au

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [22]:
pip install numpy evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 884.7 kB/s eta 0:00:00


In [23]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [24]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [25]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [27]:
!nvidia-smi

Sun Jan  7 11:34:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              26W /  70W |    571MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.104600,1.042302,0.541400
2,0.831400,0.993684,0.581400
3,0.514000,1.170075,0.584400


TrainOutput(global_step=1875, training_loss=0.8693060974121094, metrics={'train_runtime': 1858.4152, 'train_samples_per_second': 8.071, 'train_steps_per_second': 1.009, 'total_flos': 3946772136960000.0, 'train_loss': 0.8693060974121094, 'epoch': 3.0})

In [30]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [31]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.2778185606002808,
 'eval_accuracy': 0.55,
 'eval_runtime': 3.2206,
 'eval_samples_per_second': 31.05,
 'eval_steps_per_second': 4.037,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [32]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [33]:
trainer.save_state()

## Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少

## 保存文件到Google Drive

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
!pwd

/content


In [36]:
%mkdir /content/drive/MyDrive/AI/FineTune0107

In [38]:
 !cp -r models /content/drive/MyDrive/AI/FineTune0107

In [39]:
!ls -l /content/drive/MyDrive/AI/FineTune0107/models/bert-base-cased

total 8
drwx------ 2 root root 4096 Jan  7 12:16 finetuned-trainer
drwx------ 6 root root 4096 Jan  7 12:16 test_trainer


In [40]:
!ls -l -h /content/models/bert-base-cased/finetuned-trainer

total 414M
-rw-r--r-- 1 root root  955 Jan  7 12:11 config.json
-rw-r--r-- 1 root root 414M Jan  7 12:11 model.safetensors
-rw-r--r-- 1 root root 4.5K Jan  7 12:11 training_args.bin


In [41]:
!ls -l -h /content/models/bert-base-cased/test_trainer/runs

total 16K
-rw-r--r-- 1 root root 8.5K Jan  7 12:05 events.out.tfevents.1704627247.16697bc1d0f6.1393.0
-rw-r--r-- 1 root root  411 Jan  7 12:10 events.out.tfevents.1704629456.16697bc1d0f6.1393.1


In [42]:
!ls -l -h /content/models/bert-base-cased/test_trainer

total 20K
drwxr-xr-x 2 root root 4.0K Jan  7 11:49 checkpoint-1000
drwxr-xr-x 2 root root 4.0K Jan  7 11:57 checkpoint-1500
drwxr-xr-x 2 root root 4.0K Jan  7 11:40 checkpoint-500
drwxr-xr-x 2 root root 4.0K Jan  7 12:10 runs
-rw-r--r-- 1 root root 3.7K Jan  7 12:11 trainer_state.json


In [43]:
!du -h /content/models/

1.3G	/content/models/bert-base-cased/test_trainer/checkpoint-1000
1.3G	/content/models/bert-base-cased/test_trainer/checkpoint-1500
20K	/content/models/bert-base-cased/test_trainer/runs
1.3G	/content/models/bert-base-cased/test_trainer/checkpoint-500
3.7G	/content/models/bert-base-cased/test_trainer
414M	/content/models/bert-base-cased/finetuned-trainer
4.1G	/content/models/bert-base-cased
4.1G	/content/models/
